In [6]:
# Run this with the environment "encore3". It has R 3.6 installed, such that it can run icluster

import itertools
import subprocess
import pandas as pd
import os
import numpy as np
from pathlib import Path
import pathlib
import multiprocessing as mp

def get_parameters():
    files = [
        '/local/DESMOND2_data_simulated/simulated/A/A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv',
        '/local/DESMOND2_data_simulated/simulated/A/A.n_genes=50,m=4,std=1,overlap=no.exprs_z.tsv',
        '/local/DESMOND2_data_simulated/simulated/A/A.n_genes=500,m=4,std=1,overlap=no.exprs_z.tsv',
        '/local/DESMOND2_data_simulated/simulated/B/B.n_genes=5,m=4,std=1,overlap=yes.exprs_z.tsv',
        '/local/DESMOND2_data_simulated/simulated/B/B.n_genes=50,m=4,std=1,overlap=yes.exprs_z.tsv',
        '/local/DESMOND2_data_simulated/simulated/B/B.n_genes=500,m=4,std=1,overlap=yes.exprs_z.tsv',
        '/local/DESMOND2_data_simulated/simulated/C/C.n_genes=5,m=4,std=1,overlap=yes.exprs_z.tsv',
        '/local/DESMOND2_data_simulated/simulated/C/C.n_genes=50,m=4,std=1,overlap=yes.exprs_z.tsv',
        '/local/DESMOND2_data_simulated/simulated/C/C.n_genes=500,m=4,std=1,overlap=yes.exprs_z.tsv',
        
        
    ]
    n_cluster_list = list(range(4, 11, 2))
    type_list = ["gaussian","multinomial"] # 'binomial', 'poisson' throws errors
    burnin_list = np.linspace(200, 500, 2)
    draw_list = np.linspace(200, 500, 2)
    lambda_list = [1, 5, 10, 100, 'null']
    maxiter_list = [200]
    sdev_list = np.linspace(0.001, 0.2, 5)
    eps_list = np.linspace(1.0e-3, 1.0e-5, 5)
    seeds = np.arange(1, 6)

    combinations = []
    for file in files:
        for n_cluster in n_cluster_list:
            for type in type_list:
                for n_burnin in burnin_list:
                    for n_draw in draw_list:
                        for lambda_n in lambda_list:
                            for maxiter in maxiter_list:
                                for sdev in sdev_list:
                                    for eps in eps_list:
                                        for seed in [1]:
                                            combinations.append({
                                                'exprs_file':file,
                                                'lambda_n':lambda_n,
                                                'n_cluster':n_cluster,
                                                'lambda_scale':1, 
                                                'iter_max':maxiter,
                                                'eps':eps, 
                                                'type':type, 
                                                'burnin_n':n_burnin, 
                                                'draw_n':n_draw, 
                                                'sdev':sdev,
                                                'seed': seed
                                            })
    return combinations

def read_runtime():
    runtime_string = open('iClusterPlusRuntime.txt', 'r').read().strip()
    runtime = float(runtime_string)
    os.remove('iClusterPlusRuntime.txt')
    return runtime

def run(exprs_file, lambda_n='null', n_cluster=4, lambda_scale=1, iter_max=20, eps=1.0e-4, type='gaussian', burnin_n=100, draw_n=200, sdev=0.05, seed=1):
    # Given K, the number of cluster is K+1.
    n_cluster -= 1

    directory = "/iClusterPlusCluster/exprs_file={}/lambda_n={}/n_cluster={}/lambda_scale={}/iter_max={}/eps={}/type={}/burnin_n={}/draw_n={}/sdev={}/seed={}".format(
        exprs_file.split('/')[-1],
        lambda_n,
        n_cluster,
        lambda_scale,
        iter_max,
        eps,
        type,
        burnin_n,
        draw_n,
        sdev,
        seed
    )
    directory = str(pathlib.Path().resolve()) + directory

    Path(directory).mkdir(parents=True, exist_ok=True)
    
    if Path(os.path.join(directory, 'result.csv')).is_file():
        print(os.path.join(directory, 'result.csv'), 'already exists. Skipping...')
        return
    
    ### check another time for runs with more max_iter
    directory = "/iClusterPlusCluster/exprs_file={}/lambda_n={}/n_cluster={}/lambda_scale={}/iter_max={}/eps={}/type={}/burnin_n={}/draw_n={}/sdev={}/seed={}".format(
        exprs_file.split('/')[-1],
        lambda_n,
        n_cluster,
        lambda_scale,
        10000,
        eps,
        type,
        burnin_n,
        draw_n,
        sdev,
        seed
    )
    directory = str(pathlib.Path().resolve()) + directory
    if Path(os.path.join(directory, 'result.csv')).is_file():
        print(os.path.join(directory, 'result.csv'), 'already exists. Skipping...')
        return
    
    # this saves the result to a file
    # time is measured inside the R script
    command = fr'/usr/bin/Rscript ./methods/iClusterPlus.R {exprs_file} {lambda_n} {n_cluster} {lambda_scale} {iter_max} {eps} {type} {burnin_n} {draw_n} {sdev} {directory} {seed}'
    print(command)
    subprocess.Popen(command, shell=True).wait()
    return #format_output(n_cluster), read_runtime()



In [7]:
combinations = get_parameters()
len(combinations)

36000

In [ ]:
def worker_wrapper(kwargs):
    # to support kwargs in mp map
    return run(**kwargs)

with mp.Pool(20) as pool:
    pool.map(worker_wrapper, combinations)


/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=1/n_cluster=3/lambda_scale=1/iter_max=10000/eps=0.001/type=gaussian/burnin_n=200.0/draw_n=200.0/sdev=0.001/seed=1/result.csv/usr/bin/Rscript ./methods/iClusterPlus.R /local/DESMOND2_data_simulated/simulated/A/A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv 5 3 1 200 0.001 multinomial 500.0 500.0 0.001 /data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=5/n_cluster=3/lambda_scale=1/iter_max=10000/eps=0.001/type=multinomial/burnin_n=500.0/draw_n=500.0/sdev=0.001/seed=1 1/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=null/n_cluster=5/lambda_scale=1/iter_max=10000/eps=0.001/type=gaussian/burnin_n=500.0/draw_n=200.0/sdev=0.001/seed=1/res

 /data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=50,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=100/n_cluster=9/lambda_scale=1/iter_max=10000/eps=0.0007525/type=gaussian/burnin_n=200.0/draw_n=500.0/sdev=0.001/seed=1/result.csv /data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=500,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=null/n_cluster=3/lambda_scale=1/iter_max=10000/eps=0.001/type=gaussian/burnin_n=200.0/draw_n=200.0/sdev=0.001/seed=1/result.csvalready exists. Skipping...
/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=100/n_cluster=3/lambda_scale=1/iter_max=10000/eps=0.0007525/type=gaussian/burnin_n=500.0/draw_n=500.0/sdev=0.001/seed=1/result.csv already exists. Skipping...

 /data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluste

/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=50,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=10/n_cluster=3/lambda_scale=1/iter_max=10000/eps=0.000505/type=gaussian/burnin_n=200.0/draw_n=200.0/sdev=0.001/seed=1/result.csv /data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=5/n_cluster=9/lambda_scale=1/iter_max=10000/eps=0.000505/type=gaussian/burnin_n=200.0/draw_n=500.0/sdev=0.001/seed=1/result.csv

/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=50,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=5/n_cluster=5/lambda_scale=1/iter_max=10000/eps=0.0007525/type=gaussian/burnin_n=500.0/draw_n=500.0/sdev=0.05075/seed=1/result.csv/usr/bin/Rscript ./methods/iClusterPlus.R /local/DESMOND2_data_simulated/simulated/A/A.n_genes=500,m=4,std=1,overlap=no.exprs_z.tsv null 3 1 200 0.0

/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=1/n_cluster=3/lambda_scale=1/iter_max=10000/eps=1e-05/type=gaussian/burnin_n=200.0/draw_n=200.0/sdev=0.05075/seed=1/result.csv/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=1/n_cluster=7/lambda_scale=1/iter_max=10000/eps=0.0007525/type=gaussian/burnin_n=500.0/draw_n=200.0/sdev=0.1005/seed=1/result.csv  already exists. Skipping...
already exists. Skipping.../data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=1/n_cluster=3/lambda_scale=1/iter_max=10000/eps=0.001/type=gaussian/burnin_n=200.0/draw_n=200.0/sdev=0.1005/seed=1/result.csv already exists. Skipping.../data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClu

Loading required package: parallel
Loading required package: data.table
Loading required package: parallel
Loading required package: data.table
Loading required package: parallel
Loading required package: parallel
Loading required package: parallel
Loading required package: data.table
Loading required package: data.table
Loading required package: parallel
Loading required package: data.table
Loading required package: parallel
Loading required package: data.table
Loading required package: data.table
Loading required package: parallel
Loading required package: parallel
Loading required package: parallel
Loading required package: parallel
Loading required package: data.table
Loading required package: data.table
Loading required package: parallel
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table


...

already exists. Skipping...
/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=100/n_cluster=3/lambda_scale=1/iter_max=10000/eps=0.001/type=gaussian/burnin_n=200.0/draw_n=200.0/sdev=0.001/seed=1/result.csv already exists. Skipping...
/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=100/n_cluster=3/lambda_scale=1/iter_max=10000/eps=0.0007525/type=gaussian/burnin_n=200.0/draw_n=200.0/sdev=0.001/seed=1/result.csv already exists. Skipping...
/data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=5,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=100/n_cluster=3/lambda_scale=1/iter_max=10000/eps=0.000505/type=gaussian/burnin_n=200.0/draw_n=200.0/sdev=0.001/seed=1/result.csv already exists. Skipping...
/usr/bin/Rscript ./methods/iClusterPlu

Loading required package: parallel
Loading required package: data.table


100  points of lambdas are used to tune parameters.
Begin parallel computation
/usr/bin/Rscript ./methods/iClusterPlus.R /local/DESMOND2_data_simulated/simulated/A/A.n_genes=500,m=4,std=1,overlap=no.exprs_z.tsv null 3 1 200 1e-05 gaussian 200.0 200.0 0.001 /data/home/bba1401/Projects/encore/DESMOND2/evaluation/factorization/iClusterPlusCluster/exprs_file=A.n_genes=500,m=4,std=1,overlap=no.exprs_z.tsv/lambda_n=null/n_cluster=3/lambda_scale=1/iter_max=10000/eps=1e-05/type=gaussian/burnin_n=200.0/draw_n=200.0/sdev=0.001/seed=1 1
End parallel computation


There were 50 or more warnings (use warnings() to see the first 50)
Loading required package: parallel
Loading required package: data.table


50  points of lambdas are used to tune parameters.
Begin parallel computation
